# 1.1 

In [1]:
import time
import os
from urllib.request import urlopen
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
from datetime import date 

In [2]:
driver_option = webdriver.ChromeOptions()
driver_option.add_argument('--headless')
# driver_option.add_argument('--incognito')
chromedriver_path = '/Users/ellpeeaxe/Desktop/SMU/Y4S1/SA/chromedriver' # Change this to your own chromedriver path!

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, chrome_options=driver_option)

In [3]:
def get_posturls(username):
    
    browser = create_webdriver()
    # Open the website
    browser.get('https://www.instagram.com/'+username)

    # Get the number of posts
    post_num_element = browser.find_element_by_class_name('g47SY')
    post_num_html = post_num_element.get_attribute('innerHTML')

    # Retrieve number of posts
    soup = BeautifulSoup(post_num_html, 'html.parser')
    num_posts = int(soup.get_text().replace(',', ''))

    urls = []
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    has_more = scroll(1,browser,last_height)

    while len(urls) < num_posts:   

        url_element = browser.find_elements_by_css_selector('div.v1Nh3.kIKUG._bz0w')

        for url in url_element:
            url = url.get_attribute('innerHTML')
            soup = BeautifulSoup(url, 'html.parser')
            urls.append(soup.find('a')['href'])

        urls = list(dict.fromkeys(urls))
        print(len(urls))
        has_more = scroll(1,browser,last_height)
        
    print(username + " " + str(len(urls)))

    browser.close()

    return urls

In [4]:
def scroll(times,browser,last_height):
    SCROLL_PAUSE_TIME = 3

    scrolling = times
    while scrolling > 0:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            return False
        last_height = new_height
        scrolling -= 1
    return True

In [5]:
def scrape_likescomments(urls):
    posts = []
    dates = []
    likes = []
    comment_count = []
    commenters = []
    comments = []
    
    for url in list(urls):
        url = 'http://instagram.com' + url
        print(url)
        page = urlopen(url).read()
        soup = BeautifulSoup(page, features="html.parser")

        string = soup.find("meta",  property="og:description")['content']
        string = string.split(" ")
        num_likes = string[0]
        num_comments = string[2]

        browser = create_webdriver()
        browser.get(url)
        
        date = browser.find_element_by_css_selector('time._1o9PC.Nzb55').get_attribute('datetime')
        print(date)

        hasLoadMore = True
        while hasLoadMore:
            time.sleep(2)
            try:
                browser.find_element_by_css_selector('button.dCJp8.afkep > span.glyphsSpriteCircle_add__outline__24__grey_9.u-__7').click()
            except:
                hasLoadMore = False

        texts = browser.find_elements_by_class_name('C4VMK')
       
        for comment in texts:
            posts.append(url)
            dates.append(date)
            likes.append(num_likes)
            comment_count.append(num_comments)
            try:
                comments.append(comment.text.split("\n")[1])
                commenters.append(comment.text.split("\n")[0])
            except:
                commenters.append(" ")
                comments.append(" ")

        browser.close()

    df = pd.DataFrame({'url': posts, 
                       'num_likes': likes, 
                       'date' : dates,
                       'num_comment': comment_count, 
                       'commenter': commenters, 
                       'comment': comments})
    return df
    

In [ ]:
if not os.path.exists(os.getcwd()+"/data/LikesComments/"):
    os.makedirs(os.getcwd()+"/data/LikesComments/")
    
for username in open("usernames.txt", "r"):
    urls = get_posturls(username.strip())
    df = scrape_likescomments(urls)
    df['username'] = username.strip()
    
    display(df)
    
    df.to_csv(os.getcwd()+"/data/LikesComments/"+username.strip()+"-"+str(date.today())+'.txt',sep='\t',index = False,encoding='utf-8')


In [ ]:
username = "seedlysg"
urls = get_posturls(username.strip())
df = scrape_likescomments(urls)
df['username'] = username.strip()

df.to_csv(os.getcwd()+"/data/LikesComments/"+username.strip()+"-"+str(date.today())+'.txt',sep='\t',index = False,encoding='utf-8')



In [ ]:
display(df)